In [79]:
import torch
import torchvision

MOTOR = 0.35
TRUN = 0.3

# Alexnet
PATH = '../models/AlexNet_S233M_W57.01M.pth'
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

# # MobileNetV2
# PATH = '../models/MobileNetV2_S 13.6M_W2.23M.pth'
# model = torchvision.models.mobilenet_v2(pretrained=False)
# model.classifier[1] = torch.nn.Linear(in_features=1280, out_features=2, bias=True)

# # RestNet18
# PATH = '../models/ResNet18_S44.7M _W11.18M.pth'
# model = torchvision.models.resnet18(pretrained=False)
# model.fc = torch.nn.Linear(in_features=512, out_features=2, bias=True)

# # squeezenet
# PATH = '../models/SqueezeNet_S 4.79M_W0.74M.pth'
# model = torchvision.models.squeezenet1_0(pretrained=False)
# model.classifier[1] = torch.nn.Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1))

model.load_state_dict(torch.load(PATH))
device = torch.device('cuda')
model = model.to(device)

In [84]:
import cv2
import numpy as np
import torch.nn.functional as F
import time
from jetbot import Robot
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
from cameraX import CameraX

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

pre_direction = True # True is Left, Flase Right
pre_state = False # True Blocked, False Free
def update(change):
    global blocked_slider, robot
    global pre_direction, pre_state
    start = time.time()
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    prob_blocked = float(y.flatten()[0])
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(MOTOR)
        if pre_state == True: # Blocked
            pre_direction = False if pre_direction else True # 反轉方向
        pre_state = False # Free
    else:
        if pre_direction:
            robot.left(TRUN)
        else:
            robot.right(TRUN + 0.1)
        pre_state = True # Blocked
    time.sleep(0.001)
#     print('pre_direction:', pre_direction)
    
    # 計算FPS
#     fps = 1 / (time.time() - start)
#     cv2.putText(image, 'FPS: {:.0f}'.format(fps), (10, 210), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255), 1, cv2.LINE_AA)
#     print('\rFPS: {:.0f}'.format(fps), end='')
    

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])
normalize = torchvision.transforms.Normalize(mean, stdev)

# camera = Camera.instance(width=224, height=224, fps=20)
cap = None
camera = CameraX(fps=20)

image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(widgets.HBox([image, blocked_slider]))
robot = Robot()

Shape: 224.0 224.0 FPS: 20.0
Buffer: 0.0


In [85]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [86]:
import time

camera.unobserve(update, names='value')
time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
robot.stop()

camera_link.unlink()  # don't stream to browser (will still run camera)
camera_link.link()  # stream to browser (wont run camera)
camera.stop()